# NO LONELY COLOR DETECTOR
In this file we have three sections:  
   1. **Main functions needed:** Cell that includes the main functions and imports that will be needed in this notebook.  
   2. **Going through all the graphs of a file:** Cell that can be used to go through all the files saved in a g6 file. Before using this cell make sure that the variable 'folderPath' is defined as needed.  
   3. **Checking an specific graph:** Cell that can be used to check an specific graph. To do so, we have to make sure to update the variable 'g6_string' with the graph6 format of the graph that we want to check.

# Main functions needed

In [ ]:
import os
import time
import supportFunctions as sf

def checkPerfectMatchings(G):
    """
    Method that generates all perfect matchings, color the graph and calls the function
    that checks each one of them.
    
    PARAMETERS:
        G: Graph object
    
    RETURNS:
        True if finds a perfect matching that acts as a no lonely coloring
        False if not
    """
    for perfectMatching in G.perfect_matchings():
        # We color the graph
        G_prime = G.copy()
        for u, v in perfectMatching:
            G_prime.set_edge_label(u, v, 1)

        if isNoLonelyColor_ForPerfectMatchings(G_prime):
            return True

    return False


def isNoLonelyColor_ForPerfectMatchings(G):
    """
    Method that vertifies a colored graph with a perfect matching by calling a DFS
    method for each vertex of the graph.
    
    PARAMETERS:
        G: Graph which has the edges of the perfect matching colored with the label 1.
             The rest of the edges have label 0
    
    RETURNS:
        True if the matching is a no lonely coloring
        False if not
    """
    visited = set()
    for v in G.vertices():
        if v not in visited:
            visited.add(v)
            if not isNoLonelyColorDFS_ForPerfectMatchings(G, v, v, visited, set(), 0):
                return False

    return True

    
def isNoLonelyColorDFS_ForPerfectMatchings(G, currentNode, finalNode, visited = set(), path = set(),
                                           numEdgesMatching = 0):
    """
    DFS method that goes through all the cycles that contain the vertex finalNode.
    
    PARAMETERS:
        G:                Graph which has the edges of the perfect matching colored with the label 1.
                            The rest of the edges have label 0
        currentNode:      Vertex that we are visiting at the moment
        finalNode:        Vertex where the cycle ends (and begins)
        visited:          set of vertices whose cycles have already been checked
        path:             set that contains the vertices that uses the path
        numEdgesMatching: number of vertices of the matching that are in the path
    
    RETURNS:
        True if all the cycles that go through finalNode are okay
        False if not there is a cycle that contains a lonely color edge
    """
    if currentNode not in path:
        path.add(currentNode)

        for nei in G.neighbors(currentNode):
            if nei not in visited and nei not in path: # To not use vertices that are already fully checked
                label = G.edge_label(currentNode, nei)
                aux_numEdgesMatching = numEdgesMatching + (1 if label == 1 else 0)

                if aux_numEdgesMatching > 1:
                    # If we find two edges of the matching in a cycle, it won't be a lonely color cycle
                    continue 
                elif not isNoLonelyColorDFS_ForPerfectMatchings(G, nei, finalNode, visited, path, aux_numEdgesMatching):
                    return False

            elif nei == finalNode and len(path) > 2:
                # We have found a cycle
                label = G.edge_label(currentNode, nei)
                aux_numEdgesMatching = numEdgesMatching + (1 if label == 1 else 0)

                if aux_numEdgesMatching == 1:
                    # We have found a cycle with a lonely color
                    return False

        path.remove(currentNode)

    return True


def checkAllColorings(G):
    """
    Functiong that calls the backtracking funcion that goes through all the colorings in the
    search of a no lonely coloring.
    
    PARAMETERS:
        G: Graph
    
    RETURNS:
        True if it finds a no lonely coloring in G.
        False if not
    """
    return checkColorings_backtrack(G.copy(), G.copy(), Graph(), G.edges()[0], 1, False, set(), 0)


def checkColorings_backtrack(G, G_aux, G_coloredEdges, edge, color, colorIsOkay, statesChecked,
                             lenMonochromaticCycle):
    """
    Backtracking function that searchs a no lonely coloring of G.
    
    PARAMETERS:
        G:                     Graph that we want to color
        G_aux:                 Graph that contains the edges that are not colored with "color"
        G_coloredEdges:        Graph that contains the colored edges
        edge:                  Edge that is going to be colored next
        color:                 Color that we will use to color "edge"
        colorIsOkay:           Boolean that tells us that, once we color "edge", if the edges colored
                                 with "color" satisfy the needed conditions to be a no lonely color
        statesChecked:         Set that contains the colorings that we have already discarded.
        lenMonochromaticCycle: If we find a monochromatic cycle, it contains the length. If not it is 0.
    
    RETURNS:
        True if it finds a no lonely coloring in G.
        False if not
    """
    u, v, label = edge[0], edge[1], edge[2]
    
    # If the edge is uncolored
    if label == 0:
        # We color the edge and update all the graphs
        G.set_edge_label(u, v, color)
        G_aux.delete_edge(u, v)
        G_coloredEdges.add_edge(u, v, color)
        
        # If this was the last edge to paint and the coloring is okay, we end
        if len(G_coloredEdges.edges()) == len(G.edges()) and colorIsOkay: return True
        
        # DEFINING edges_missing --> WE WANT TO KNOW WHICH EDGES SHOULD BE COLORED NEXT
        # We search for lonely uncolored edges in cycles
        if len(G_coloredEdges.edges()) != len(G.edges()): # If G is fully colored, there is no need
            checkNonColoredCycles, uncoloredEdges = sf.searchOfLonelyEdgesByColor(G, 0)
        else: checkNonColoredCycles = True
        
        if not checkNonColoredCycles:
            # uncoloredEdges contains a lonely uncolored edge in a cycle
            edges_missing = uncoloredEdges
        else:
            # Else we search for a lonely edge with color "color"
            checkCyclesColor, uncoloredEdges = sf.searchOfLonelyEdgesByColor(G, color)
            
            if not checkCyclesColor:
                # uncoloredEdges contains the edges of the cycle that contains a lonely edge
                edges_missing = uncoloredEdges
            else:
                # If we have not found a lonely edge of color "color"...
                if colorIsOkay:
                    # We continue coloring with another color
                    if checkColorings_backtrack(G, G.copy(), G_coloredEdges, sf.findNextEdge(G),
                                                color + 1, False, statesChecked, 0):
                        return True
                
                # If we have no candidates, we just try to color all the uncolored edges
                edges_missing = G_aux.edges()
        
        # WE EXPLORE THE CANDIDATES TO COLOR
        for edge_to_color in edges_missing:
            u_to_color, v_to_color, label_to_color = edge_to_color[0], edge_to_color[1], edge_to_color[2]

            if label_to_color == 0:                
                G_coloredEdges.add_edge(u_to_color, v_to_color, color)
                
                # If we have not discarded this state before
                if tuple(G_coloredEdges.edges()) not in statesChecked:
                    # If the vertices do not have three adjacent edges of color "color"
                    if sf.isDegreeColorGood(G_coloredEdges, edge_to_color, color):
                        lenMonochromaticCycle_currentEdge = sf.searchMonochromaticCyclesInEdge(G_coloredEdges,
                                                                                    edge_to_color, color)
                        
                        # If the monochromatic cycles have the same length
                        if (
                            lenMonochromaticCycle_currentEdge == 0
                            or lenMonochromaticCycle == 0
                            or lenMonochromaticCycle_currentEdge == lenMonochromaticCycle
                        ):
                            if lenMonochromaticCycle == 0:
                                # We have not found a previous monochromatic cycle. So, we save the new value
                                lenMonochromaticCycle = lenMonochromaticCycle_currentEdge
                            
                            # We update G_aux
                            G_aux.delete_edge(u_to_color, v_to_color)
                            
                            # If G_aux is disconnected and G_colored edges has both endpoints
                            # of each edge of color "color" in different components
                            if (
                                not G_aux.is_connected()
                                and sf.areEndpointsInDifferentComponents(G_coloredEdges,
                                                                      G_aux.connected_components(), color)
                            ):
                                # The edges of color "color" satisfy the necessary conditions
                                # --> colorIsOkay = True
                                if checkColorings_backtrack(G, G_aux, G_coloredEdges, edge_to_color,
                                                            color, True, statesChecked, lenMonochromaticCycle):
                                    return True
                            
                            # If not, we continue considering that the color is not okay
                            # --> colorIsOkay = False
                            if checkColorings_backtrack(G, G_aux, G_coloredEdges, edge_to_color,
                                                        color, False, statesChecked, lenMonochromaticCycle):
                                return True

                            # We restore G_aux
                            G_aux.add_edge(u_to_color, v_to_color, 0)
                
                # We restore G_coloredEdges
                G_coloredEdges.delete_edge(u_to_color, v_to_color)
        
        # NONE OF THE edges_missing CAN BE COLORED WITH color
        # To save memory, we do not save the cases that are the only option after a previous state
        if len(edges_missing) != 1: statesChecked.add(tuple(G_coloredEdges.edges()))
        
        # We restore the graphs
        G.set_edge_label(u, v, 0)
        G_aux.add_edge(u, v, 0)
        G_coloredEdges.delete_edge(u, v)

    return False

# Going through all the graphs of a file

In [ ]:
def isNoLonelyColor(G, f, g6_string, check, perfectMatchingChecking):
    """
    Main functions that checks if a given graph has a no lonely coloring.
    
    PARAMETERS:
        G:                       Graph
        f:                       File where we write the candidates for not being no lonely color
                                   graphs (the ones that do not have a no lonely coloring using
                                   perfect matchings)
        g6_string:               string with g6 format of the graph G
        check:                   Boolean that tells us if we have to save the graphs in f or not.
        perfectMatchingChecking: Boolean that tells us if we are going to check the perfect
                                   matchings first or if we can go directly to check all colorings.
    
    RETURNS:
        True if G is a no lonely coloring or if we are done saving the graph into f.
        False if the graph is not a no lonely coloring.
    """
    if perfectMatchingChecking:
        if not checkPerfectMatchings(G):
            # We only check all the possible colorings if we don't save the information of each graph
            if check:   # We save the graphs that can not be colored using perfect matchings
                with open(output_file, 'a') as f:
                    f.write(g6_string)

            else:
                # We check all the possibilities
                return checkAllColorings(G)
    
    else:
        # We don't check the perfect matchings because it have been checked in a previous execution
        return checkAllColorings(G)

    return True


# WRITE THE FOLDER THAT YOU WANT TO USE
folderPath = './'
nameOutputFile = 'graphs_to_check.g6'

output_file = os.path.join(folderPath, nameOutputFile)

# If the file does not exist, we creat it
os.makedirs(folderPath, exist_ok=True)
if not os.path.exists(output_file):
    open(output_file, 'w').close()

# We go through the files in the folder "folderPath"
for file in os.listdir(folderPath):
    if file.endswith('.g6') or file.endswith('.graph6'):
        filePath = os.path.join(folderPath, file)

        check = input(f"Do you want to open the file: {file}? (y/n)")
        
        perfectMatchingChecking = True
        if check == 'y':
            # If we are opening the file "nameOutputFile", we will not check the perfect matchings another time
            if file == nameOutputFile: perfectMatchingChecking = False
            
            if perfectMatchingChecking:
                # If we select yes, we will only go through the perfect matchings and save the ones that do not have a solution
                check = input(f"Do you want to save the possible lonely colors in {output_file}? (y/n)")
                if check =='y': check = True
                else: check = False
            
            start_global = time.time()            
            f = open(filePath, "r")
            lines = f.readlines()

            for i, line in enumerate(lines):
                start = time.time()
                g6_string = line

                try:
                    G = Graph(g6_string)
                except Exception as e:
                    print(f"Exception: {e}")
                    print(f"Graph: {g6_string}")
                    continue

                # We set the labels of the edges as 0
                for u, v, _ in G.edges():
                    G.set_edge_label(u, v, 0)

                if not isNoLonelyColor(G, output_file, g6_string, check, perfectMatchingChecking):
                    # If we have not found a no lonely coloring...
                    print(f"Graph {i} in {file}: FALSE")
                    print(g6_string)

                    # Draw the SageMath graph
                    G.show()
                    # RECOMMENDED TO UNCOMMENT IF SEARCHING FOR PROHIBITED GRAPHS OF GIRHT 4 OR GREATER IN LARGE FILES
                    #sys.exit("Program interrupted: PROHIBITED GRAPH FOUND.")

                end = time.time()
                print(f"{g6_string} checked in {end-start}s")

            end_global = time.time()
            print(f"{file} checked in {end_global-start_global}s")

# Checking an specific graph

In [ ]:
def isNoLonelyColor(G):
    """
    Main functions that checks if a given graph has a no lonely coloring.
    
    PARAMETERS:
        G: Graph
    
    RETURNS:
        True if G is a no lonely coloring or if we are done saving the graph into f.
        False if the graph is not a no lonely coloring.
    """
    if not checkPerfectMatchings(G):
        # We check all the possibilities
        return checkAllColorings(G)

    return True


# WRITE GRAPH IN FORMAT g6
g6_string = r"KsP@PGWCOH?R"

try:
    G = Graph(g6_string)
except:
    print("ERROR: Introduce a valid graph in g6 format.")
    G = Graph()

for u, v, _ in G.edges():  # We set the edges of the graph as 0
    G.set_edge_label(u, v, 0)

start = time.time()
if not isNoLonelyColor(G):
    print("No lonely color graph found.")

    # Draw the SageMath graph
    G.show()

end = time.time()
print(f"{g6_string} checked in {end-start}s")